# LangChain 🦜 ~ Free Web Search 

## Objectives : 

    1. Langchain is bad at maths
    ----------------------------
    Let's prove it
    Let's fix it
    
    2. SERPAPI needs tokens
    -----------------------
    We can avoid it
    
    3. Solve a deprecation warning for LLMMathChain
    -----------------------------------------------
    New Syntax


Code from: https://github.com/RGGH/LangChain_101/blob/main/free_web_search.ipynb

In [1]:
#!pip install duckduckgo-search
#!pip install wikipedia

### Imports

In [2]:
import os
from langchain import hub
from langchain_openai import OpenAI
from langchain.agents import Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import create_react_agent, AgentExecutor

In [3]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

### LLM

In [4]:
llm = OpenAI(temperature=0)

### Tools

In [5]:
search = DuckDuckGoSearchRun()
wikipedia = WikipediaAPIWrapper()


# Web Search Tool
search_tool = Tool(
    name="Web Search",
    func=search.run,
    description="A useful tool for searching the Internet to find information on world events, years, dates, issues, etc. Worth using for general topics. Use precise questions.",
)

# Wikipedia Tool
wikipedia_tool = Tool(
    name="Wikipedia",
    func=wikipedia.run,
    description="A useful tool for searching the Internet to find information on world events, issues, dates, years, etc. Worth using for general topics. Use precise questions.",
)


from langchain import LLMMathChain

#llm_math_chain = LLMMathChain(llm=llm, verbose=True) <- Deprecated !!!

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
math_tool = Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions.",
    )

### Agent

In [6]:
prompt = hub.pull("hwchase17/react")

In [7]:
agent = create_react_agent(
    tools = [search_tool,wikipedia_tool,math_tool],
    llm=llm,
    prompt=prompt
)

In [8]:
query = "search for King Charles III' current age in years and then calculate the cube root of that number to 1 decimal"

### Run

In [9]:
tools = [search_tool,wikipedia_tool,math_tool]
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
agent_executor.invoke(input={"input": query})



> Entering new AgentExecutor chain...
 I should use Wikipedia to find King Charles III's birthdate and then use a calculator to calculate the cube root of his current age.
Action: Wikipedia
Action Input: King Charles IIIPage: Charles III
Summary: Charles III (Charles Philip Arthur George; born 14 November 1948) is King of the United Kingdom and the 14 other Commonwealth realms.Charles was born in Buckingham Palace during the reign of his maternal grandfather, King George VI, and became heir apparent when his mother, Queen Elizabeth II, acceded to the throne in 1952. He was created Prince of Wales in 1958 and his investiture was held in 1969. He was educated at Cheam School and Gordonstoun, and later spent six months at the Timbertop campus of Geelong Grammar School in Victoria, Australia. After earning a history degree from the University of Cambridge, Charles served in the Royal Air Force and the Royal Navy from 1971 to 1976. In 1981 he married Lady Diana Spencer. They had two sons,

{'input': "search for King Charles III' current age in years and then calculate the cube root of that number to 1 decimal",
 'output': '4.2'}

In [11]:
4.198336453808407**3

73.99999999999997